# HBN Fixes

This notebook documents the diffferent fixes for HBN

## DWI (CODE 32)



In [2]:
def insert_in_json(fname, datadict, dry_run=True):
    
    with open(fname, 'r') as read_file:
        dat = json.load(read_file)
    dat = json.load(fname)
    dat.update(dat)
    if dry_run:
        print(dat)
    else:
        print('updating')

In [3]:
import pandas as pd
import json

def get_target_from_code(dataset, code):
    
    df = pd.read_csv('/cbica/projects/RBC/flywheel_curation/RBC/PennLINC/Validation/CUBIC_Curation/{}_validation.csv'.format(dataset))

    df[['subject', 'session', 'folder', 'filename']] = df['files'].str.split('/', expand=True).loc[:,1:4]

    res = df.loc[(df['code'] == code), ['files', 'subject', 'session', 'filename']]
    return res

In [27]:
get_target_from_code('HBN', 32)

,files,subject,session,filename
9646,/sub-NDARFG953RRK/ses-HBNsiteCBIC/dwi/sub-NDAR...,sub-NDARFG953RRK,ses-HBNsiteCBIC,sub-NDARFG953RRK_ses-HBNsiteCBIC_acq-64dir_dwi...
9647,/sub-NDARJZ526HN3/ses-HBNsiteRU/dwi/sub-NDARJZ...,sub-NDARJZ526HN3,ses-HBNsiteRU,sub-NDARJZ526HN3_ses-HBNsiteRU_acq-64dir_dwi.n...
9648,/sub-NDARTN487KDK/ses-HBNsiteRU/dwi/sub-NDARTN...,sub-NDARTN487KDK,ses-HBNsiteRU,sub-NDARTN487KDK_ses-HBNsiteRU_acq-64dir_dwi.n...


The DWI data for this person does exist; we can download it again from flywheel:

In [20]:
%%bash

fw export bids --project RBC_HBN --subject sub-NDARFG953RRK --data-type dwi /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/

INFO: Verify download directory exists
INFO: Processing project files
INFO: Processing session files
INFO: Processing acquisition files
INFO: Downloading project files
INFO: Downloading session files
INFO: Downloading acquisition files
INFO: Creating sidecar files
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/dataset_description.json

Flywheel CLI 12.4.1 build 5bdff2e3c1d8c8ffd82e4995f3ad195b7618c359 on 2020-08-18 08:26



In [22]:
%%bash

fw export bids --project RBC_HBN --subject sub-NDARJZ526HN3 --data-type dwi /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/

INFO: Verify download directory exists
INFO: Processing project files
INFO: Processing session files
INFO: Processing acquisition files
INFO: Downloading project files
INFO: Downloading session files
INFO: Downloading acquisition files
INFO: Downloading acquisition file: sub-NDARJZ526HN3_ses-HBNsiteRU_acq-64dir_dwi.bvec
INFO: Creating sidecar files
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/dataset_description.json
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/sub-NDARJZ526HN3/ses-HBNsiteRU/dwi/sub-NDARJZ526HN3_ses-HBNsiteRU_acq-64dir_dwi.bvec

Flywheel CLI 12.4.1 build 5bdff2e3c1d8c8ffd82e4995f3ad195b7618c359 on 2020-08-18 08:26



In [23]:
%%bash

fw export bids --project RBC_HBN --subject sub-NDARTN487KDK /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/

INFO: Verify download directory exists
INFO: Processing project files
INFO: Processing session files
INFO: Processing acquisition files
INFO: Downloading project files
INFO: Downloading session files
INFO: Downloading acquisition files
INFO: Downloading acquisition file: sub-NDARTN487KDK_ses-HBNsiteRU_acq-64dir_dwi.bvec
INFO: Creating sidecar files
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/dataset_description.json
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/sub-NDARTN487KDK/ses-HBNsiteRU/dwi/sub-NDARTN487KDK_ses-HBNsiteRU_acq-64dir_dwi.bvec

Flywheel CLI 12.4.1 build 5bdff2e3c1d8c8ffd82e4995f3ad195b7618c359 on 2020-08-18 08:26



---

## ECHO_TIME 1&2 (CODE 15)

This section is updated for fieldmaps with no echo times defined

In [28]:
get_target_from_code('HBN', 15)

,files,subject,session,filename
9583,/sub-NDARRE333EKT/ses-HBNsiteSI/fmap/sub-NDARR...,sub-NDARRE333EKT,ses-HBNsiteSI,sub-NDARRE333EKT_ses-HBNsiteSI_phasediff.nii.gz


This participant has the data in the json on Flywheel. Let's fetch:

In [43]:
import flywheel

client = flywheel.Client()

proj = client.projects.find_first('label=RBC_HBN')
subject = proj.subjects.find_first('label="sub-NDARRE333EKT"')

In [45]:
sessions = subject.sessions()[0]

In [48]:
acquisitions = sessions.acquisitions()

In [50]:
phasediff = [ac for ac in acquisitions if 'phasediff' in ac.label]

In [52]:
phasediff = client.get(phasediff[0].id)

In [54]:
info = phasediff.files[0].info

In [56]:
json_data = info.copy()

del json_data['BIDS']

In [57]:
json_data

{'AcquisitionMatrixPE': 64,
 'BaseResolution': 64,
 'ConversionSoftware': 'dcm2niix',
 'ConversionSoftwareVersion': 'v1.0.20171204 (OpenJPEG build) GCC4.8.4',
 'DeviceSerialNumber': '26121',
 'DwellTime': 3e-05,
 'EchoNumber': 2,
 'EchoTime': 0.00804,
 'FlipAngle': 90,
 'ImageOrientationPatientDICOM': [1, 0, 0, 0, 1, 0],
 'ImageType': ['ORIGINAL', 'PRIMARY', 'P', 'ND'],
 'InPlanePhaseEncodingDirectionDICOM': 'COL',
 'MRAcquisitionType': '2D',
 'MagneticFieldStrength': 1.5,
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'Avanto',
 'Modality': 'MR',
 'PartialFourier': 1,
 'PatientPosition': 'HFS',
 'PercentPhaseFOV': 100,
 'PhaseEncodingDirection': 'j-',
 'PhaseEncodingSteps': 64,
 'PhaseResolution': 1,
 'PixelBandwidth': 260,
 'ProcedureStepDescription': 'CMI_HBN',
 'ProtocolName': 'Field_Map',
 'PulseSequenceDetails': '%SiemensSeq%_gre_field_mapping',
 'ReceiveCoilName': '32Ch_Head',
 'ReconMatrixPE': 64,
 'RepetitionTime': 0.492,
 'SAR': 0.244346,
 'ScanningSequence': 'GR',
 '

In [58]:
def write_json(data, filename='data.json'): 
    with open(filename,'w') as f: 
        json.dump(data, f, indent=4)

In [59]:
outputfile = '/cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/sub-NDARRE333EKT/ses-HBNsiteSI/fmap/sub-NDARRE333EKT_ses-HBNsiteSI_phasediff.json'

write_json(json_data, outputfile)

Scratch the above, just download the subject again:

In [60]:
%%bash

fw export bids --project RBC_HBN --subject sub-NDARRE333EKT /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/

INFO: Verify download directory exists
INFO: Processing project files
INFO: Processing session files
INFO: Processing acquisition files
INFO: Downloading project files
INFO: Downloading session files
INFO: Downloading acquisition files
INFO: Downloading acquisition file: sub-NDARRE333EKT_ses-HBNsiteSI_task-rest_bold.nii.gz
INFO: Downloading acquisition file: sub-NDARRE333EKT_ses-HBNsiteSI_T1w.nii.gz
INFO: Creating sidecar files
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/dataset_description.json
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/sub-NDARRE333EKT/ses-HBNsiteSI/func/sub-NDARRE333EKT_ses-HBNsiteSI_task-rest_bold.nii.gz
INFO: Creating sidecar file: /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/sub-NDARRE333EKT/ses-HBNsiteSI/anat/sub-NDARRE333EKT_ses-HBNsiteSI_T1w.nii.gz

Flywheel CLI 12.4.1 build 5bdff2e3c1d8c8ffd82e4995f3ad195b7618c359 on 2020-08-18 

---

## EMPTY FILE CODE 99

Assuming these subjects just didn't download from flywheel correctly:

In [3]:
get_target_from_code('HBN', 99)

,files,subject,session,filename
9879,/sub-NDARFG953RRK/ses-HBNsiteCBIC/dwi/sub-NDAR...,sub-NDARFG953RRK,ses-HBNsiteCBIC,sub-NDARFG953RRK_ses-HBNsiteCBIC_acq-64dir_dwi...
9880,/sub-NDARJZ526HN3/ses-HBNsiteRU/dwi/sub-NDARJZ...,sub-NDARJZ526HN3,ses-HBNsiteRU,sub-NDARJZ526HN3_ses-HBNsiteRU_acq-64dir_dwi.bval
9881,/sub-NDARTN487KDK/ses-HBNsiteRU/dwi/sub-NDARTN...,sub-NDARTN487KDK,ses-HBNsiteRU,sub-NDARTN487KDK_ses-HBNsiteRU_acq-64dir_dwi.bval
9882,/sub-NDARAB055BPR/ses-HBNsiteRU/func/sub-NDARA...,sub-NDARAB055BPR,ses-HBNsiteRU,sub-NDARAB055BPR_ses-HBNsiteRU_task-movieTP_bo...
9883,/sub-NDARAX277ATU/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARAX277ATU,ses-HBNsiteCBIC,sub-NDARAX277ATU_ses-HBNsiteCBIC_acq-fMRI_dir-...
9884,/sub-NDARBH992ARB/ses-HBNsiteCBIC/func/sub-NDA...,sub-NDARBH992ARB,ses-HBNsiteCBIC,sub-NDARBH992ARB_ses-HBNsiteCBIC_task-peer_run...
9885,/sub-NDARBJ159HXB/ses-HBNsiteRU/func/sub-NDARB...,sub-NDARBJ159HXB,ses-HBNsiteRU,sub-NDARBJ159HXB_ses-HBNsiteRU_task-movieTP_bo...
9886,/sub-NDARBV167RMU/ses-HBNsiteCBIC/func/sub-NDA...,sub-NDARBV167RMU,ses-HBNsiteCBIC,sub-NDARBV167RMU_ses-HBNsiteCBIC_task-peer_run...
9887,/sub-NDARCK815WZR/ses-HBNsiteCBIC/func/sub-NDA...,sub-NDARCK815WZR,ses-HBNsiteCBIC,sub-NDARCK815WZR_ses-HBNsiteCBIC_task-movieTP_...
9888,/sub-NDARDE502TEU/ses-HBNsiteRU/func/sub-NDARD...,sub-NDARDE502TEU,ses-HBNsiteRU,sub-NDARDE502TEU_ses-HBNsiteRU_task-movieTP_bo...


In [7]:
df = get_target_from_code('HBN', 99)

' '.join(list(df.subject))

'sub-NDARFG953RRK sub-NDARJZ526HN3 sub-NDARTN487KDK sub-NDARAB055BPR sub-NDARAX277ATU sub-NDARBH992ARB sub-NDARBJ159HXB sub-NDARBV167RMU sub-NDARCK815WZR sub-NDARDE502TEU sub-NDARDH270YR5 sub-NDARDU566NUY sub-NDAREC777ADZ sub-NDAREK686WN1 sub-NDARGD425PTN sub-NDARGY542TJ5 sub-NDARHX934KJ5 sub-NDARJF322BN4 sub-NDARKZ312XR1 sub-NDARLU939YX4 sub-NDARLW774VY1 sub-NDARMY254BGN sub-NDARMY456WML sub-NDARNB427ARE sub-NDARNK055HVH sub-NDARNM783ZVV sub-NDARPH152GU1 sub-NDARPL440BX5 sub-NDARPR799VX4 sub-NDARRE333EKT sub-NDARTG937LDA sub-NDARVG777EUZ sub-NDARVL190JA8 sub-NDARVN899GWP sub-NDARVN920VHJ sub-NDARWF422UK3 sub-NDARYF272EDC sub-NDARYZ693DE2 sub-NDARZT940RZG'

In [ ]:
%%bash

subs=( sub-NDARFG953RRK sub-NDARJZ526HN3 sub-NDARTN487KDK sub-NDARAB055BPR sub-NDARAX277ATU sub-NDARBH992ARB sub-NDARBJ159HXB sub-NDARBV167RMU sub-NDARCK815WZR sub-NDARDE502TEU sub-NDARDH270YR5 sub-NDARDU566NUY sub-NDAREC777ADZ sub-NDAREK686WN1 sub-NDARGD425PTN sub-NDARGY542TJ5 sub-NDARHX934KJ5 sub-NDARJF322BN4 sub-NDARKZ312XR1 sub-NDARLU939YX4 sub-NDARLW774VY1 sub-NDARMY254BGN sub-NDARMY456WML sub-NDARNB427ARE sub-NDARNK055HVH sub-NDARNM783ZVV sub-NDARPH152GU1 sub-NDARPL440BX5 sub-NDARPR799VX4 sub-NDARRE333EKT sub-NDARTG937LDA sub-NDARVG777EUZ sub-NDARVL190JA8 sub-NDARVN899GWP sub-NDARVN920VHJ sub-NDARWF422UK3 sub-NDARYF272EDC sub-NDARYZ693DE2 sub-NDARZT940RZG )

for i in "${subs[@]}"; do qsub-run fw export bids --project RBC_HBN --subject $i /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/; done

In [5]:
get_target_from_code('HBN', 99).iloc[-1,:].filename

'sub-NDARZT940RZG_ses-HBNsiteCBIC_acq-fMRI_dir-PA_epi.nii.gz'

This subject does seem to have something weird going on with this image; there is poor contrast; not sure what to do here. We'll probably have to just list these people and make sure we circle back to them

---

## PHASE ENCODING CODE 18

Let's make sure the phase encoding directions are there:

In [7]:
get_target_from_code('HBN', 18)

,files,subject,session,filename
9584,/sub-NDARCK815WZR/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARCK815WZR,ses-HBNsiteCBIC,sub-NDARCK815WZR_ses-HBNsiteCBIC_acq-dwi_dir-A...
9585,/sub-NDARCK815WZR/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARCK815WZR,ses-HBNsiteCBIC,sub-NDARCK815WZR_ses-HBNsiteCBIC_acq-fMRI_dir-...
9586,/sub-NDARCK815WZR/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARCK815WZR,ses-HBNsiteCBIC,sub-NDARCK815WZR_ses-HBNsiteCBIC_acq-dwi_dir-P...
9587,/sub-NDARCK815WZR/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARCK815WZR,ses-HBNsiteCBIC,sub-NDARCK815WZR_ses-HBNsiteCBIC_acq-fMRI_dir-...
9588,/sub-NDARDH270YR5/ses-HBNsiteRU/fmap/sub-NDARD...,sub-NDARDH270YR5,ses-HBNsiteRU,sub-NDARDH270YR5_ses-HBNsiteRU_acq-fMRI_dir-AP...
9589,/sub-NDARFG953RRK/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARFG953RRK,ses-HBNsiteCBIC,sub-NDARFG953RRK_ses-HBNsiteCBIC_acq-dwi_dir-P...
9590,/sub-NDARFG953RRK/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARFG953RRK,ses-HBNsiteCBIC,sub-NDARFG953RRK_ses-HBNsiteCBIC_acq-dwi_dir-A...
9591,/sub-NDARFG953RRK/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARFG953RRK,ses-HBNsiteCBIC,sub-NDARFG953RRK_ses-HBNsiteCBIC_acq-fMRI_dir-...
9592,/sub-NDARFG953RRK/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARFG953RRK,ses-HBNsiteCBIC,sub-NDARFG953RRK_ses-HBNsiteCBIC_acq-fMRI_dir-...
9593,/sub-NDARGY542TJ5/ses-HBNsiteCBIC/fmap/sub-NDA...,sub-NDARGY542TJ5,ses-HBNsiteCBIC,sub-NDARGY542TJ5_ses-HBNsiteCBIC_acq-fMRI_dir-...


In [8]:
df = get_target_from_code('HBN', 18)

' '.join(list(df.subject))

'sub-NDARCK815WZR sub-NDARCK815WZR sub-NDARCK815WZR sub-NDARCK815WZR sub-NDARDH270YR5 sub-NDARFG953RRK sub-NDARFG953RRK sub-NDARFG953RRK sub-NDARFG953RRK sub-NDARGY542TJ5 sub-NDARGY542TJ5 sub-NDARGY542TJ5 sub-NDARGY542TJ5 sub-NDARJZ526HN3 sub-NDARJZ526HN3 sub-NDARJZ526HN3 sub-NDARJZ526HN3 sub-NDARPL440BX5 sub-NDARPL440BX5 sub-NDARPL440BX5 sub-NDARPL440BX5 sub-NDARRE333EKT sub-NDARTN487KDK sub-NDARTN487KDK sub-NDARTN487KDK sub-NDARTN487KDK sub-NDARVN899GWP sub-NDARVN899GWP sub-NDARVN899GWP sub-NDARVN899GWP sub-NDARZT940RZG'

In [ ]:
%%bash

subs=( sub-NDARCK815WZR sub-NDARCK815WZR sub-NDARCK815WZR sub-NDARCK815WZR sub-NDARDH270YR5 sub-NDARFG953RRK sub-NDARFG953RRK sub-NDARFG953RRK sub-NDARFG953RRK sub-NDARGY542TJ5 sub-NDARGY542TJ5 sub-NDARGY542TJ5 sub-NDARGY542TJ5 sub-NDARJZ526HN3 sub-NDARJZ526HN3 sub-NDARJZ526HN3 sub-NDARJZ526HN3 sub-NDARPL440BX5 sub-NDARPL440BX5 sub-NDARPL440BX5 sub-NDARPL440BX5 sub-NDARRE333EKT sub-NDARTN487KDK sub-NDARTN487KDK sub-NDARTN487KDK sub-NDARTN487KDK sub-NDARVN899GWP sub-NDARVN899GWP sub-NDARVN899GWP sub-NDARVN899GWP sub-NDARZT940RZG )
for i in "${subs[@]}"; do qsub-run fw export bids --project RBC_HBN --subject $i /cbica/projects/RBC/flywheel_curation/RBC/data/bids_datasets/RBC_HBN/; done

This may have fixed it; cursory glance with the bids-validator doesn't show this error